In [ ]:
# Define the time variable
time_var = ncfile.variables['time']

# Convert the time variable to a human-readable format
time = nc.num2date(time_var[:], time_var.units)

# Specify the group
group_name = 'ZX_LIDAR_WLBZ_6'
group = ncfile.groups[group_name]

# Extract variables
wind_speed = group.variables['wind_speed'][:]
wind_from_direction = group.variables['wind_from_direction'][:]
wind_from_direction_availability = group.variables['wind_from_direction_availability'][:]
turbulence_intensity = group.variables['turbulence_intensity'][:]

# Create DataFrames for each height
height_indices = range(wind_speed.shape[3])  # height is the 4th dimension
dataframes = {}

for height_idx in height_indices:
    df = pd.DataFrame({
        'time': time,
        'wind_speed': wind_speed[:, 0, 0, height_idx],
        'wind_from_direction': wind_from_direction[:, 0, 0, height_idx],
        'wind_from_direction_availability': wind_from_direction_availability[:, 0, 0, height_idx],
        'turbulence_intensity': turbulence_intensity[:, 0, 0, height_idx]
    })
    
    # Set time as the index
    df.set_index('time', inplace=True)
    
    # Store the DataFrame in the dictionary
    dataframes[f'height_{height_idx}'] = df

# Heights in meters for each DataFrame
heights = [14, 42, 94, 140, 200, 250]

# Assign DataFrames to variables dynamically
for idx, height in enumerate(heights):
    globals()[f'height_{height}'] = dataframes[f'height_{idx}']

height_140

In [ ]:
# Height of interest 3. place in the 6. dimension of the ds = dataset
heights = [14, 42, 94, 140, 200, 250]
height_idx = 3

# Function to process buoy data for the specific height
def process_buoy_data(file_path, group_name):
    # Open the nc file
    ds = Dataset(file_path)
    
    # Extract the time variable
    time = ds.variables['time']
    time = nc.num2date(time[:], time.units)
    
    # Extract the group
    group = ds.groups[group_name]
    
    # Extract variables
    wind_speed = group.variables['wind_speed'][:]
    wind_from_direction = group.variables['wind_from_direction'][:]
    wind_from_direction_availability = group.variables['wind_from_direction_availability'][:]
    turbulence_intensity = group.variables['turbulence_intensity'][:]
    
    # Create DataFrame
    df = pd.DataFrame({
        'time': time,
        'wind_speed': wind_speed[:, 0, 0, height_idx],
        'wind_from_direction': wind_from_direction[:, 0, 0, height_idx],
        'wind_from_direction_availability': wind_from_direction_availability[:, 0, 0, height_idx],
        'turbulence_intensity': turbulence_intensity[:, 0, 0, height_idx]
    })
    
    # Set time as the index
    #df.set_index('time', inplace=True)
    
    return df

# Process buoy 6 data
bouy6_data = process_buoy_data(file_path, 'ZX_LIDAR_WLBZ_6')

# Process buoy 2 data
#bouy2_data = process_buoy_data(bouy2_path, 'ZX_LIDAR_WLBZ_2')

bouy6_data

In [ ]:
# Dictionary to store filtered DataFrames and statistics
filtered_dataframes = {}
nan_stats = {}

# Filter NaN values and calculate statistics
for idx, height in enumerate(heights):
    height_key = f'height_{height}'
    df = dataframes[f'height_{idx}']
    
    original_length = len(df)
    filtered_df = df.dropna()
    filtered_length = len(filtered_df)
    
    nan_count = original_length - filtered_length
    nan_percentage = (nan_count / original_length) * 100
    
    # Store the filtered DataFrame
    filtered_dataframes[height_key] = filtered_df
    
    # Store the statistics
    nan_stats[height_key] = {
        'filtered_nan_count': nan_count,
        'filtered_nan_percentage': nan_percentage
    }
# Assign filtered DataFrames to variables 
for height in heights:
    globals()[f'filter_height{height}'] = filtered_dataframes[f'height_{height}']

# Print NaN statistics for all heights
print("\nNaN Statistics for all heights:")
for height, stats in nan_stats.items():
    print(f"Height {height}: {stats['filtered_nan_count']} NaN values filtered ({stats['filtered_nan_percentage']:.2f}%)")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure the index is converted to datetime for all heights
for height in heights:
    df = globals()[f'filter_height{height}']
    df.index = pd.to_datetime([pd.Timestamp(t.isoformat()) for t in df.index])

# Define the date range
start_date = '2022-03-03'
end_date = '2022-03-17'

# Plot wind speed for all heights in one plot, within the specified date range
plt.figure(figsize=(14, 8))

for height in heights:
    df = globals()[f'filter_height{height}']
    df_time_selected = df.loc[start_date:end_date]
    plt.plot(df_time_selected.index, df_time_selected['wind_speed'], label=f'Height {height}m')

plt.xlabel('Time')
plt.ylabel('Wind Speed (m/s)')
plt.title('Wind Speed at Different Heights')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Define the date range
start_date = '2022-03-03'
end_date = '2023-03-03'

# Filter DataFrames for the specified date range
dfs_filtered = {}
for height in heights:
    df = globals()[f'filter_height{height}']
    dfs_filtered[height] = df.loc[start_date:end_date]

# Create bins with 0.5 m/s bin width
bin_width = 0.5
max_wind_speed = max([df['wind_speed'].max() for df in dfs_filtered.values()])
bins = np.arange(0, max_wind_speed + bin_width, bin_width)

height_pairs = [(14, 42), (94, 140), (200, 250)]

# Plot histograms for wind speed at the defined height pairs in subplots
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(14, 18), sharex=True)

for ax, (height1, height2) in zip(axes, height_pairs):
    df_filtered1 = dfs_filtered[height1]
    df_filtered2 = dfs_filtered[height2]
    
    ax.hist(df_filtered1['wind_speed'], bins=bins, alpha=0.5, label=f'Height {height1}m')
    ax.hist(df_filtered2['wind_speed'], bins=bins, alpha=0.5, label=f'Height {height2}m')
    
    ax.set_xlabel('Wind Speed (m/s)')
    ax.set_ylabel('Frequency')
    ax.set_title(f'Wind Speed Distribution at Heights {height1}m and {height2}m')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
from windrose import WindroseAxes

# Function to plot wind rose
def plot_wind_rose(ax, df, height):
    ax.bar(df['wind_from_direction'], df['wind_speed'], normed=True, opening=0.8, edgecolor='white')
    ax.set_title(f'Wind Rose at Height {height}m')
    ax.set_legend()

# Create subplots for wind roses
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(18, 18), subplot_kw=dict(projection='windrose'))

# Plot wind roses for the defined heights
for ax, height in zip(axes.flatten(), heights):
    df = globals()[f'filter_height{height}']
    plot_wind_rose(ax, df, height)

plt.tight_layout()
plt.show()


In [ ]:
# Specify the main group
subgroup_name = ncfile.groups['METEO_WLBZ_6']

# Extract time variable from the main group
try:
    time_var = nc.num2date(subgroup_name.variables['time'][:], subgroup_name.variables['time'].units)
except KeyError:
    time_var = None

# Function to convert NetCDF subgroup to DataFrame
def nc_subgroup_to_df(subgroup, time_var=None):
    data_dict = {}
    if time_var is not None:
        data_dict['time'] = time_var
    
    for variable_name in subgroup.variables:
        variable = subgroup.variables[variable_name]
        data_dict[variable_name] = variable[:].flatten()  # Flatten to 1D array if necessary
    
    df = pd.DataFrame(data_dict)
    if 'time' in df.columns:
        df.set_index('time', inplace=True)
    return df

# Iterate over the subgroups in the main group and convert them to DataFrames
dfs = {}
for subgroup_name in subgroup_name.groups:
    subgroup = subgroup_name.groups[subgroup_name]
    df = nc_subgroup_to_df(subgroup, time_var)
    dfs[subgroup_name] = df
# Filter out NaN values from each DataFrame
filtered_dfs = {name: df.dropna() for name, df in dfs.items()}

vaisala_df = dfs['Vaisala Weather Station']
airmar_df = dfs['Airmar Weather Station']

airmar_df



In [ ]:
vaisala_df

## Wind farm field 

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Define the extent of the field and the surrounding area
field_extent = [5.10, 6.50, 54.10, 54.40]  # [min_lon, max_lon, min_lat, max_lat]
surrounding_extent = [2, 8, 51, 55]  # Larger area to encompass Netherlands, Germany, etc.


# Met mast positions
met_mast1_lon, met_mast1_lat = 5.521, 54.4 #Bouy6
met_mast2_lon, met_mast2_lat = 5.792, 54.5 #Bouy2

# Create a figure and axes
proj1=ccrs.LambertConformal(central_longitude=10.0, central_latitude=51, standard_parallels=(48, 54)) 
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1, projection=proj1)
ax.set_extent(surrounding_extent, crs=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, color='lightgray')

# Plot the square representing the farm field and the met masts 
min_lon, max_lon, min_lat, max_lat = field_extent
ax.plot([min_lon, max_lon, max_lon, min_lon, min_lon], 
        [min_lat, min_lat, max_lat, max_lat, min_lat], 
        color='red', linewidth=2, transform=ccrs.PlateCarree(), label='Wind fields N9.1-3')

ax.plot(met_mast1_lon, met_mast1_lat, 'bo', markersize=10, transform=ccrs.PlateCarree(), label='Bouy6')
ax.plot(met_mast2_lon, met_mast2_lat, 'go', markersize=10, transform=ccrs.PlateCarree(), label='Bouy2')
ax.legend()
ax.set_title('Field Area with Met Mast Positions')
plt.show()


In [ ]:
 fig, axes = plt.subplots(1, 2, figsize=(13, 8), sharey=True, sharex=True)

axes[0].plot(time6_filtered[4], ws6_150m)
axes[0].set_title('Wind Speed for interpolated 150 m at Buoy 6')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Wind Speed (m/s)')

axes[1].plot(time2_filtered[4], ws2_150m)
axes[1].set_title('Wind Speed for interpolated 150 m at Buoy 2')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Wind Speed (m/s)')


# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 8), sharey=True, sharex=True)

axes[0].plot(time6_filtered[4], wd6_150m)
axes[0].set_title('Wind Direction for interpolated 150 m at Buoy 6')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Direction in Degrees')

axes[1].plot(time2_filtered[4], wd2_150m)
axes[1].set_title('Wind Direction for interpolated 150 m at Buoy 2')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Direction in Degrees')


# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
# Plotting time6 against ws_150m 
plt.figure(figsize=(10, 6))
plt.plot(time6_filtered[4], ws6_150m, label='Interpolated Data at 150m')
plt.xlabel('Time')
plt.ylabel('Interpolated Values')
plt.title('Time vs Interpolated Data at 150m')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def interpolate_dataframe(df, col1, col2, height1, height2, target_height):
    # Calculate the interpolation factor
    factor = (target_height - height1) / (height2 - height1)
    # Perform the interpolation
    df[f'{col1}_{target_height}m'] = df[col1] + factor * (df[col2] - df[col1])
    return df

# Interpolating and adding the results to the dataframe for buoy6
filtered_buoy6 = interpolate_dataframe(filtered_buoy6, 'wind_speed_140m', 'wind_speed_200m', 140, 200, 150)
filtered_buoy6 = interpolate_dataframe(filtered_buoy6, 'wind_direction_140m', 'wind_direction_200m', 140, 200, 150)

# Interpolating and adding the results to the dataframe for buoy2
filtered_buoy2 = interpolate_dataframe(filtered_buoy2, 'wind_speed_140m', 'wind_speed_200m', 140, 200, 150)
filtered_buoy2 = interpolate_dataframe(filtered_buoy2, 'wind_direction_140m', 'wind_direction_200m', 140, 200, 150)

# Check the shapes
print(filtered_buoy6[f'wind_speed_140m_150m'].shape)
print(filtered_buoy6[f'wind_direction_140m_150m'].shape)
print()
print(filtered_buoy2[f'wind_speed_140m_150m'].shape)
print(filtered_buoy2[f'wind_direction_140m_150m'].shape)
